## 챕터 12: 내장 자료형 상속과 다중 상속

- 다음 두 가지 특징을 집중적으로 알아보자
    - 내장 자료형 상속의 위험성
    - 다중 상속과 메서드 결정 순서
    

### 내장 자료형의 상속은 까다롭다

**공식적으로 CPython은 내장 자료형의 서브클래스에서 오버라이드한 메서드가 언제 호출되는지, 혹은 호출되지 않는지에 대한 명확한 규칙을 정의하지 않는다. 일반적으로 서브클래스에서 오버라이드한 메서드는 같은 객체의 다른 내장 메서드에 의해 결코 호출되지 않는다. 예를 들어 dict의 서브클래스에서 오버라이드한 __getitem__()메서드는 내장된 get()과 같은 메서드에 의해 호출되지 않는다.** 

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [9]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [10]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [11]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [6]:
# AnswerDict의 __getitem__()을 지나치는 dict.update()
class AnswerDict(dict):
    def __getitem__(self, key): #Answer.__getitem__()은 키와 무관하게 42를 반환한다.
        return 42
    
ad = AnswerDict(a='foo') #ad는 ('a', 'foo') 키-값 쌍으로 채운 AnswerDict객체다.
ad['a'] # 예상한대로 ad['a']는 42를 반환한다.

42

In [7]:
d = {}
d.update(ad) #d는 평범한 dict객체며, 여기에서는 ad 객체를 이용해서 갱신한다.
d['a'] #dict.update() 메서드는 오버라이드된 AnswerDict.__getitem__()메서드를 무시한다.

'foo'

In [8]:
d

{'a': 'foo'}

In [12]:
import collections 

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [13]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [14]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [15]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [16]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [17]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [18]:
d = {}
d.update(ad)
d['a']

42

In [19]:
d

{'a': 42}

### 다중 상속과 메서드 결정 순서

In [20]:
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong', self)
        
class C(A):
    def pong(self):
        print('PONG', self)
        
class D(B,C):
    def ping(self):
        super().ping()
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [21]:
from diamond import *
d = D()
d.pong()

ModuleNotFoundError: No module named 'diamond'

In [22]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [23]:
from diamond import D
d = D()
d.ping()

ModuleNotFoundError: No module named 'diamond'

In [24]:
from diamond import D
d = D()
d.pingpong()

ModuleNotFoundError: No module named 'diamond'

In [25]:
bool.__mro__

(bool, int, object)

In [26]:
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

In [27]:
print_mro(bool)

bool, int, object


In [28]:
from frenchdeck2 import FrenchDeck2
print_mro(FrenchDeck2)

ModuleNotFoundError: No module named 'frenchdeck2'

In [29]:
import numbers
print_mro(numbers.Integral)

Integral, Rational, Real, Complex, Number, object


In [30]:
import io
print_mro(io.BytesIO)

BytesIO, _BufferedIOBase, _IOBase, object


In [31]:
print_mro(io.TextIOWrapper)

TextIOWrapper, _TextIOBase, _IOBase, object


### 실세계에서의 다중 상속

In [32]:
import tkinter
print_mro(tkinter.Toplevel)

Toplevel, BaseWidget, Misc, Wm, object


In [33]:
print_mro(tkinter.Widget)

Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [34]:
print_mro(tkinter.Button)

Button, Widget, BaseWidget, Misc, Pack, Place, Grid, object


In [35]:
print_mro(tkinter.Entry)

Entry, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, object


In [36]:
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


### 다중 상속 다루기
#### 1. 인터페이스 상속과 구현 상속을 구분한다.
    * 인터페이스 상속은 'is-a' 관계를 의미하는 서브타입을 생성한다.
    * 구현 상속은 재사용을 통해 코드 중복을 피한다.
#### 2. ABC를 이용해서 인터페이스를 명확히 한다.
#### 3. 코드를 재사용하기 위해 믹스인을 사용한다.
#### 4. 이름을 통해 믹스인임을 명확히 한다.
#### 5. ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다.
#### 6. 두 개 이상의 구상 클래스에 상속받지 않는다.
#### 7. 사용자에게 집합 클래스를 제공한다.
#### 8. 클래스 상속보다 객체 구성을 사용하라.